# Notebook to visually load some sample images and transform them

The aim is to make some easy visualization of the progress/effect of various the image transformers

## Imports

In [ ]:
import os
from typing import Dict

import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler, Subset

from image_aquisition.BasicTestDataset import BasicTestDataset
from utils.ConfigLoader import ConfigLoader
from utils.Registries import AGENT_FACTORY_REGISTRY

## Adjust/set environment variables - no fun with that


In [ ]:
%env ENV_NAME=HOLGER
os.chdir(r"C:\Users\holge\Dokumente (lokal)\CASML4SE\agentic_photography_instructor")
os.getcwd()

## Global Functions

In [ ]:
def collate_keep_size(batch):
    # Gibt einfach die Liste der einzelnen Tensoren zurück
    return batch

## Prepare and initialize DataLoader

The dataloader will provide batches of images up to a given number of images.

In [ ]:
config: Dict = ConfigLoader().load(env=os.environ["ENV_NAME"])
image_dir = config["dev"]["cloned_image_dir"]
dataset: Dataset = BasicTestDataset(image_dir)
print(f"Number of images in {image_dir}: {len(dataset)}")

# sampler to return only 10 random samples from the dataset. Not sure what replacement=True means
num_samples = 10
rndSampler = RandomSampler(dataset, replacement=True, num_samples=num_samples)
subsetSampler = SequentialSampler(Subset(dataset, range(num_samples)))

# collate_keep_size sorgt dafür, dass die Bilder ihre Originalgrösse behalten können. Standatdmässig müssten alle Bilder die gleiche Grösse haben, was wir aber nicht wollen, da wir hin diesem Moment die Bilder untransformiert lassen wollen.
dataloader = DataLoader(dataset, batch_size=2, collate_fn=collate_keep_size, sampler=subsetSampler)

## Prepare transformers and transformation agents

The transformation agents will prcess the images coming from the data loader. We will for each image render the original image the label and the transformed image. By this we are able to get a first impression what the various transformers will produce

In [ ]:
# For every batch
for batch in dataloader:
    # for every image in the batch
    for image, path, filename in batch:
        # for every factory (that produces a list of transformation agents
        image_rgb = image[..., ::-1]
        for factory_name in AGENT_FACTORY_REGISTRY.keys():
            for agent in AGENT_FACTORY_REGISTRY.get(factory_name).create_agents():
                image_clone = image.copy()
                transformed_image, label = agent.transform(image_clone)
                # Visualisierung
                plt.figure(figsize=(10, 5))
                plt.suptitle(f"{filename} | Label: {label}", fontsize=14)
                plt.subplot(1, 2, 1)
                plt.imshow(image_rgb)
                plt.title("Original")
                plt.axis('off')
                plt.subplot(1, 2, 2)
                if transformed_image.ndim == 2:
                    # grayscaled image has only 2 dimensions. We need to render accordingly
                    plt.imshow(transformed_image, cmap="gray")
                else:
                    transformed_image_rgb = transformed_image[..., ::-1]
                    plt.imshow(transformed_image_rgb)
                plt.title("Transformed")
                plt.axis('off')
                plt.tight_layout()
                plt.show()
